# Agent-based Evaluation
Chatbot Evaluation as Multi-agent Simuation

## Define Chatbot

In [17]:
from typing import List

import ollama


# This is flexible, but you can define your agent here, or call your agent API here.
def my_chat_bot(messages: List[dict]) -> dict:
    system_message = {
        "role": "system",
        "content": "You are a customer support agent for an airline.",
    }
    messages = [system_message] + messages
    completion = ollama.chat(
        messages=messages, model="Llama3.2-Korean"
    )

    formatted_response = {
        'content': completion.message.content,
        'role': completion.message.role,
        'function_call': None,
        'tool_calls': completion.message.tool_calls
    }
    return formatted_response

In [18]:
my_chat_bot([{"role": "user", "content": "hi!"}])

{'content': 'Hello! How can I assist you today? Are you looking to book a flight, make changes to your existing reservation, or have any other questions about our services?',
 'role': 'assistant',
 'function_call': None,
 'tool_calls': None}

## Define Simulated User

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama

system_prompt_template = """You are a customer of an airline company. \
You are interacting with a user who is a customer support person. \

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
instructions = """Your name is Harrison. You are trying to get a refund for the trip you took to Alaska. \
You want them to give you ALL the money back. \
This trip happened 5 years ago."""

prompt = prompt.partial(name="Harrison", instructions=instructions)

model = ChatOllama(model="Llama3.2-Korean")

simulated_user = prompt | model

In [20]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="Hi! How can I help you?")]
simulated_user.invoke({"messages": messages})

AIMessage(content="I'm Harrison and I booked a flight to Alaska five years ago. Unfortunately, the trip was cancelled due to unforeseen circumstances, and I'd like to request a full refund for my entire ticket price. Can we discuss this further?", additional_kwargs={}, response_metadata={'model': 'Llama3.2-Korean', 'created_at': '2024-12-09T07:26:27.807684Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1997132917, 'load_duration': 33805042, 'prompt_eval_count': 112, 'prompt_eval_duration': 579000000, 'eval_count': 48, 'eval_duration': 1374000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-124e8e6d-1fc5-4d50-a750-9bb685782dd5-0', usage_metadata={'input_tokens': 112, 'output_tokens': 48, 'total_tokens': 160})

## Define the agent simulation
1. The 2 nodes: the simulated user and the chatbot
2. The graph with a conditional stopping criterion

### Define nodes

In [21]:
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage


def chat_bot_node(state):
    messages = state["messages"]
    # Convert from LangChain format to the OpenAI format, which our chatbot function expects.
    messages = [convert_message_to_dict(m) for m in messages]
    # Call the chat bot
    chat_bot_response = my_chat_bot(messages)
    # Respond with an AI Message
    return {"messages": [AIMessage(content=chat_bot_response["content"])]}

In [22]:
def _swap_roles(messages):
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages


def simulated_user_node(state):
    messages = state["messages"]
    # Swap roles of messages
    new_messages = _swap_roles(messages)
    # Call the simulated user
    response = simulated_user.invoke({"messages": new_messages})
    # This response is an AI message - we need to flip this to be a human message
    return {"messages": [HumanMessage(content=response.content)]}

### Define edges
- Either we continue and call the customer support bot
- Or we finish and the conversation it over

In [23]:
def should_continue(state):
    messages = state["messages"]
    if len(messages) > 6:
        return "end"
    elif messages[-1].content == "FINISHED":
        return "end"
    else:
        return "continue"

### Define graph

In [24]:
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)
# Every response from  your chat bot will automatically go to the
# simulated user
graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    # If the finish criteria are met, we will stop the simulation,
    # otherwise, the virtual user's message will be sent to your chat bot
    {
        "end": END,
        "continue": "chat_bot",
    },
)
# The input will first go to your chat bot
graph_builder.add_edge(START, "chat_bot")
simulation = graph_builder.compile()

## Run simulation

In [25]:
for chunk in simulation.stream({"messages": []}):
    # Print out all events aside from the final end chunk
    if END not in chunk:
        print(chunk)
        print("----")

{'chat_bot': {'messages': [AIMessage(content="<|start_header_id|>assistant<|end_header_id|>\n\nHello! I'm happy to help you with any questions or concerns you may have about your flight or travel experience with our airline. What can I assist you with today?", additional_kwargs={}, response_metadata={}, id='a1779b83-d674-4cd0-ac84-997bf26ff810')]}}
----
{'user': {'messages': [HumanMessage(content="Hi, my name is Harrison and I'm calling regarding a refund for the trip I took to Alaska five years ago. Can you please look into this for me?", additional_kwargs={}, response_metadata={}, id='4c004e74-898c-4a8e-ba6a-878a1cdcdfa3')]}}
----
{'chat_bot': {'messages': [AIMessage(content="Hello Harrison! Thank you for reaching out to us about your request for a refund for your trip to Alaska five years ago. I'd be happy to help you with that.\n\nTo start, could you please provide me with some details from your original booking confirmation or travel documents? This will help me locate the specifi